In [ ]:
def PullNB (reqtime, routenum, agency):
    
    import os
    import datetime
    import json
    import requests
    import calendar
    import numpy
    import pprint
    import time

    nbparams = {"a": agency, "r": routenum, "t": reqtime}

    r1 = requests.get("http://webservices.nextbus.com/service/publicJSONFeed?command=vehicleLocations", params=nbparams)
    r1.raise_for_status()

    data1 = r1.json() #assigns the json file
    VehDat = data1['vehicle'] #sorts json file per vehicle entry        

    VehRes = {} #empty dictionary
    counter = 1
    for i in VehDat: #populate dictionary using nested dictionary aka the use of [counter]

        VehRes[counter] = {}
        VehRes[counter]['id'] = i['id']
        VehRes[counter]['lat'] = i['lat']
        VehRes[counter]['lon'] = i['lon']
        VehRes[counter]['secsSinceReport'] = i['secsSinceReport']
        VehRes[counter]['rtime'] = time.time()*1000
        counter = counter + 1
    
    return VehRes

In [ ]:
def create_table():
    
    import sys
    import psycopg2
    import time
    import datetime
    import calendar
    
    conn = psycopg2.connect(host="localhost", database="VehLoc", user="postgres", password="postgres")
    year = datetime.datetime.today()
    year = year.year
    
    try:
        cur = conn.cursor()
        # After connecting to the SQL database this creates a table based on the year if it does not yet exist
        sql = "CREATE TABLE IF NOT EXISTS TESTcollection_pm_" + str(year) + "(ID REAL, Lat VARCHAR(70), Lon VARCHAR(70), RouteNum VARCHAR(70), secsSinceReport REAL, RTime BIGINT, Agency VARCHAR(70))"
        cur.execute(sql)
        # execute the Create statement

        # commit the changes to the database. Without this line nothing created will stay in the database.
        conn.commit()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
#Closes the connection to the database. Make sure to close this so the database won't get slowed down with extra connections.
            conn.close()

In [ ]:
def csv_read():
    
    import sys
    import time
    import csv
    import datetime
    import schedule
    
    #Reads CSV with list of agencies and routenums
    file = "C:/Users/LUB/testing_csv.csv"
    
    with open(file, "r") as csvfile:
        
        csvreader = csv.reader(csvfile)
        # If your csv has a header keep this as true, otherwise set to false
        is_header = 0
        #Goes through each CSV row and pulls a set agency and routenum
            
        agency_list = []
        routenum_list = []
       
        for row in csvreader:
            if is_header:
                is_header = False
                continue
            else:
            #define agency and routenum from the CSV file.
                agency = row[0]
                routenum = row[1]
                agency_list.append(agency)
                routenum_list.append(routenum)
                
    
    #Returns lists of agencies and route numbers.
    return agency_list, routenum_list

In [ ]:
def houston(status):
    
    import requests
    
    slack_token = "placeholder"
    
    StudyName = 'NextBus Vehicle Location'
    
    if status == "initialize":
        text = "{} Running".format(StudyName)
    #if status == "Read Success":
    #    text = "{} CSV Read".format(StudyName)
    if status == "success":
        text = "{} Complete".format(StudyName)
    #if status == "fail":
    #    text = "{} Failed".format(StudyName)
    if status == "error":
        text = "{} Error".format(StudyName)
    if status == "reschedule":
        text = "Reschedule {}".format(StudyName)
    
    channel = "#api_status"
    #Paremeters
    houstonparams = {"token": slack_token, "channel": channel, "as_user": "true", "text": text}

    #Slack message request line
    r1 = requests.get("https://slack.com/api/chat.postMessage?", params=houstonparams)
    r1.raise_for_status()

In [ ]:
def apicall(routenum_list, agency_list):
    global VehRes
    global rowCounter
    import psycopg2
    import time
    import csv
    import pprint
    import datetime
    import schedule
    
    houston("initialize")
    
    limit = 1000000
    delay_time = 20
    
    conn = psycopg2.connect(host="localhost",database="VehLoc", user="postgres", password="postgres")
    timer = datetime.datetime.today()
    dateref = timer.date()
    
    #Pull date format from current day to input into SQL
    day = dateref.strftime("%A")
    date = dateref.strftime("%d")
    month = dateref.strftime("%B")

    timer = datetime.datetime.today()
    timerh = timer.hour
    timerm = timer.minute
    timers = timer.second
    timer = timerh*60 + timerm
    hour_stop = timerh*60 + timerm + 120

    requests = 0
        
    while timer <= hour_stop:
        tic = time.clock()
        
        for i in range(rowCounter, len(routenum_list)):
            if requests <=limit and timer <=hour_stop:
                routenum = routenum_list[rowCounter]
                agency = agency_list[rowCounter]                
                reqtime = "now"
                #timestamp = time.strftime("%H:%M:%S")
                VehRes = PullNB(reqtime, routenum, agency)

                counter = 1
                pprint.pprint(VehRes)
                for j in range(0,len(VehRes)): #similar nested dictionary 
                    VehRes[counter]['day'] = day
                    VehRes[counter]['date'] = date
                    VehRes[counter]['month'] = month
                    VehRes[counter]['agency'] = agency
                    VehRes[counter]['routenum'] = routenum

                    year = datetime.datetime.today()
                    year = year.year
                    
                    cur = conn.cursor()
                        # execute the INSERT statement
                    sql = "INSERT INTO TESTcollection_pm_" + str(year) + "(ID, lat, lon, routenum, secsSinceReport, rtime, agency)\
                            VALUES(%(id)s, %(lat)s, %(lon)s, %(routenum)s, %(secsSinceReport)s, %(rtime)s, %(agency)s)"
                    cur.execute(sql,VehRes[counter])
                    pprint.pprint(VehRes[counter])
                    counter = counter+1
                conn.commit()
                        
                timer = datetime.datetime.today()
                timerh = timer.hour
                timerm = timer.minute
                timers = timer.second
                timer = timerh*60 + timerm 
                
                if rowCounter < len(routenum_list):
                    rowCounter += 1
            else:
                break
        if rowCounter == len(routenum_list):
            rowCounter = 0
            
        toc = time.clock()
        delay = toc-tic
        sleepTime = delay_time - delay

        #Put the program to sleep for an interval. This will delay for your delay_time and subtract how long the program ran for.
        if sleepTime > 0 and requests < limit:
            time.sleep(sleepTime)
        #Return new hour to check if it's past the stop hour.
        
    conn.close    
    houston ("success")

In [ ]:
create_table()
agency_list, routenum_list = csv_read()
global rowCounter
global VehRes
import schedule
import datetime
import time
rowCounter = 0

apicall(routenum_list, agency_list)

In [ ]:
# main function

create_table()
agency_list, routenum_list = csv_read()
global rowCounter
global VehRes
import schedule
import datetime
import time
rowCounter = 0

#Schedule the code to run at a particular time for a particular day.
schedule.every().monday.at("7:30").do(apicall, routenum_list, agency_list).tag('api')
schedule.every().monday.at("16:00").do(apicall, routenum_list, agency_list).tag('api')

schedule.every().tuesday.at("7:30").do(apicall, routenum_list, agency_list).tag('api')
schedule.every().tuesday.at("16:00").do(apicall, routenum_list, agency_list).tag('api')

schedule.every().wednesday.at("7:30").do(apicall, routenum_list, agency_list).tag('api')
schedule.every().wednesday.at("16:00").do(apicall, routenum_list, agency_list).tag('api')

schedule.every().thursday.at("7:30").do(apicall, routenum_list, agency_list).tag('api')
schedule.every().thursday.at("16:00").do(apicall, routenum_list, agency_list).tag('api')

schedule.every().friday.at("7:30").do(apicall, routenum_list, agency_list).tag('api')
schedule.every().friday.at("16:00").do(apicall, routenum_list, agency_list).tag('api')


timer = datetime.datetime.today()
timer = timer.day

while timer < 35:
    schedule.run_pending()
    time.sleep(10)
    timer = datetime.datetime.today()
    timer = timer.day
    
schedule.clear('api')
cur.close()

houston("reschedule")

print("Done")